### Add an animated speaking character to your video

**THIS NOTEBOOK IS SUPPOSED TO BE RUN ON COLAB**

This notebook uses the PyToon library to add an animated character with mouth movements to the generated video.

PyToon: https://github.com/lukerbs/pytoon/tree/main

The above library uses character images created by https://github.com/carykh/lazykh

### Why the notebook?

PyToon uses ForceAlign (https://github.com/lukerbs/forcealign), a library for forced alignment of English text to English Audio.
ForceAlign uses Pytorch's WAV2VEC2 pretrained model for acoustic feature extraction.

My setup does not have enough RAM to run the above model (I'm on WSL2 on an old PC), hence I decided to split the code and run this part on Colab (the free version, I used a T4 GPU).
You can do the same or you can modify the convert_to_video() function in order to directly generate the final result.

In [ ]:
# Colab comes with ffmpeg already installed, but you can check by running the following command
!apt-get install ffmpeg

In [ ]:
!pip install pytoon --quiet

**Important:** before running the cell below, you need to upload the following files:
- slides_content.json
- all the audio clips in MP3 format (create a folder on Colab and put them there)
- all the video clips in MP4 format (create a folder on Colab and put them there)

All these files are generated when you run the generate.py script. You'll find them in the folder you specified as output folder ('output' by default).

If you have enough resources you can avoid the for loop and pass a single MP3 file and a single MP4 file as background video.

**Note:** the FPS parameter has an impact on the resource you'll use. With FPS=24 Colab uses ~9GB of RAM and should not crash.

In [ ]:
import os
import json
import tempfile

from pytoon.animator import animate
from moviepy.editor import VideoFileClip, concatenate_videoclips

with open('./slides_content.json', 'r') as f_in:
    slides_content = json.load(f_in)

with tempfile.TemporaryDirectory() as temp_path:
    frames_path = []
    for i, slide in enumerate(slides_content):
        print(f"Processing slide {i} ...")
        # Create a Pytoon animation
        animation = animate(
            audio_file=f"./audio_clips/frame_{i}.mp3",
            transcript=slide.get('slide_narration', ''),
            fps=24
        )
        # Overlay the animation on top of another video and save as an .mp4 file
        background_video = VideoFileClip(f"./video_clips/frame_{i}.mp4")
        frame_path = os.path.join(temp_path, f"final_video_frame_{i}.mp4")
        animation.export(
            path=frame_path,
            background=background_video,
            scale=0.5
        )
        frames_path.append(frame_path)
    # Concatenate frames
    clips = []
    for path in frames_path:
        clips.append(VideoFileClip(filename=path))
    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile(
        filename=os.path.join('final_video_with_animated_character.mp4'), fps=24
    )